# Análise exploratória

**TODO:**
- (1) Tabela
- (2) Gráfico
- (3) Tabela



Solicitação: "Bom dia Seguem dados que demonstram que a transmissão de GPS dos veículos da linha 309 estava ativa e disponível para a SMTR, porém as viagens de sentido VOLTA não foram apontadas no relatório de pagamento.Favor solicitar reprocessamento destes dias, senão teremos de abrir recurso para mais de 180 viagens".

## Análise das viagens da amostra x apuradas

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [1]:
#!pip install matplotlib
import basedosdados as bd
from datetime import timedelta, datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go


pd.set_option('display.max_rows', 10)

# Paths
import os
import sys
from pathlib import Path 

# current_path = Path().resolve().parent
current_path = Path().resolve()
parent_path = current_path.parent
grandparent_path = parent_path.parent

paths = dict()
paths["scripts"] = grandparent_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *
from circular_trips import *
from gps_data import *
from treat_data import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

Importar amostra

In [2]:
# Nome do arquivo Excel
caminho_amostra = "../data/raw/Relatorio de Viagens Realizadas Sentido Volta - Linha 309 - 31.12.2022 e 01.01.2023.xlsx"
sheet1 = pd.read_excel(caminho_amostra, sheet_name=0)
sheet2 = pd.read_excel(caminho_amostra, sheet_name=1)
amostra = pd.concat([sheet1, sheet2], ignore_index=True)

# Renomear as colunas
colunas_renomeadas = {
    'Data': 'data',
    'Linha': 'servico',
    'Carro': 'id_veiculo',
    'Sentido': 'sentido',
    'Saída ': 'hora_inicio',
    'Chegada': 'hora_fim'
}

amostra.rename(columns=colunas_renomeadas, inplace=True)


# Criar as colunas `datetime_partida` e `datetime_chegada`
amostra['datetime_partida'] = amostra['data'].astype(str) + ' ' + amostra['hora_inicio'].astype(str)
amostra['datetime_chegada'] = amostra['data'].astype(str) + ' ' + amostra['hora_fim'].astype(str)

# Transformar essas colunas em tipo datetime
amostra['datetime_partida'] = pd.to_datetime(amostra['datetime_partida'])
amostra['datetime_chegada'] = pd.to_datetime(amostra['datetime_chegada'])

amostra['id_veiculo'] =  amostra['id_veiculo'].astype('str')

amostra

,data,servico,id_veiculo,sentido,hora_inicio,hora_fim,datetime_partida,datetime_chegada
0,2022-12-31,309,41380,VOLTA,04:10:00,05:08:00,2022-12-31 04:10:00,2022-12-31 05:08:00
1,2022-12-31,309,41125,VOLTA,04:20:00,05:15:00,2022-12-31 04:20:00,2022-12-31 05:15:00
2,2022-12-31,309,41054,VOLTA,04:31:00,05:34:00,2022-12-31 04:31:00,2022-12-31 05:34:00
3,2022-12-31,309,41464,VOLTA,04:41:00,05:48:00,2022-12-31 04:41:00,2022-12-31 05:48:00
4,2022-12-31,309,41310,VOLTA,04:51:00,05:45:00,2022-12-31 04:51:00,2022-12-31 05:45:00
...,...,...,...,...,...,...,...,...
176,2023-01-01,309,41125,VOLTA,18:38:00,20:00:00,2023-01-01 18:38:00,2023-01-01 20:00:00
177,2023-01-01,309,41325,VOLTA,19:00:00,20:25:00,2023-01-01 19:00:00,2023-01-01 20:25:00
178,2023-01-01,309,41147,VOLTA,19:20:00,20:27:00,2023-01-01 19:20:00,2023-01-01 20:27:00
179,2023-01-01,309,41030,VOLTA,19:39:00,20:53:00,2023-01-01 19:39:00,2023-01-01 20:53:00


In [3]:
amostra = remove_overlapping_trips(amostra)

Verificar se existe sobreposição de viagens na amostra

In [4]:
# não existe nenhuma viagem sobreposta
amostra['status'].value_counts()

Series([], Name: status, dtype: int64)

In [5]:
datas = query_values(amostra, 'data') 
id_veiculo = query_values(amostra, 'id_veiculo') 

viagem_completa = query_viagem_completa(datas, id_veiculo)


    SELECT
      data,
      SUBSTRING(id_veiculo, 2) as id_veiculo,
      servico_informado,
      sentido,
      datetime_partida,
      datetime_chegada
    FROM
      `rj-smtr.projeto_subsidio_sppo.viagem_completa`
    WHERE 
      data IN ('2022-12-31','2023-01-01')
      AND SUBSTRING(id_veiculo, 2) IN ('41380','41125','41054','41464','41310','41011','41215','41265','41198','41021','41462','41278','41172','41244','41014','41010','41257','41313','41246','41026','41409','41377','41356','41396','41428','41111','41344','41104','41448','41372','41421','41092','41465','41247','41186','41435','41004','41030','41282','41391','41296','41209','41238','41202','41147','41442','41325','41322')
    


Downloading: 100%|██████████| 350/350 [00:00<00:00, 469.17rows/s]


Execução de query de viagens completas: sucesso.


In [6]:
viagem_completa = treat_trips(viagem_completa)
amostra_classificada_completa = check_trips(amostra, viagem_completa, "Viagem identificada e já paga")
print(amostra_classificada_completa.shape[0])
amostra_classificada_completa.to_excel('./../data/treated/viagem_completa.xlsx')

Não existem casos duplicados no cruzamento de dados.
181


Conformidade

In [7]:
viagem_conformidade = query_viagem_conformidade(datas, id_veiculo)


  SELECT
    data,
    SUBSTRING(id_veiculo, 2) as id_veiculo,
    servico_informado,
    sentido,
    datetime_partida,
    datetime_chegada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
  WHERE 
    data IN ('2022-12-31','2023-01-01')
    AND SUBSTRING(id_veiculo, 2) IN ('41380','41125','41054','41464','41310','41011','41215','41265','41198','41021','41462','41278','41172','41244','41014','41010','41257','41313','41246','41026','41409','41377','41356','41396','41428','41111','41344','41104','41448','41372','41421','41092','41465','41247','41186','41435','41004','41030','41282','41391','41296','41209','41238','41202','41147','41442','41325','41322')
  


Downloading: 100%|██████████| 481/481 [00:00<00:00, 1239.69rows/s]

Execução de query de viagens conformidade: sucesso.


In [8]:
viagem_conformidade = treat_trips(viagem_conformidade)
amostra_classificada_conformidade = check_trips(amostra, viagem_conformidade, "Viagem não atendeu os critérios de conformidade")
print(amostra_classificada_conformidade.shape[0])
amostra_classificada_conformidade.to_excel('./../data/treated/viagem_conformidade.xlsx')

Não existem casos duplicados no cruzamento de dados.
181


Viagens circulares

In [9]:
amostra_circular = circular_trips(amostra_classificada_conformidade, viagem_completa, viagem_conformidade)
amostra_circular.to_excel('./../data/treated/viagem_circular.xlsx')

Verificando se existem linhas circulares.


Downloading: 100%|██████████| 5/5 [00:00<00:00, 12.02rows/s]

Execução de query de viagens planejada: sucesso.
Empty DataFrame
Columns: [data, servico_amostra, id_veiculo_amostra, sentido_amostra, hora_inicio, hora_fim, datetime_partida_amostra, datetime_chegada_amostra, status, data_apurado, id_veiculo_apurado, servico_apurado, sentido_apurado, datetime_partida_apurado, datetime_chegada_apurado, circular_dividida]
Index: []
Não existem viagens circulares divididas em shapes de ida e volta.


In [10]:
print(amostra_circular['status'].value_counts())

# Viagem não atendeu os critérios de conformidade    102
# 79 viagens não encontradas

Viagem não atendeu os critérios de conformidade    102
Name: status, dtype: int64


In [11]:
amostra_circular.to_excel('./../data/output/analise_amostra_pre_solucao.xlsx')

Todas as viagens do dia 31/12/2022 foram classificadas como sem conformidade, enquanto as viagens do dia 01/01/2023 não foram encontradas na tabela de conformidade. No dia 31/12/2022, os indicadores na tabela de conformidade indicaram que a tabela atende sim aos critérios de conformidade.

Ao verificar o modelo `projeto_subsidio_sppo` para os dois dias, foi verificado que:

- Ao rodar novamente o modelo em dev para o dia 01-01-2023, as viagens apareceram na tabela `viagem_conformidade`.

- Na tabela `viagem_completa`, foi feita uma correção do bug para o serviço, para que as viagens em conformidade e que atenderam aos critérios fossem reconhecidas. O resultado está no dataset: `rj-smtr-dev.SMTR202302003175_reprocessamento.viagem_completa`.


Reprocessamento

In [12]:
viagem_completa_reprocessada = query_viagem_completa(data = datas, id_veiculo = id_veiculo, reprocessed=True,
                                                     reprocessed_table='rj-smtr-dev.SMTR202302003175_reprocessamento.viagem_completa')


    SELECT
      data,
      SUBSTRING(id_veiculo, 2) as id_veiculo,
      servico_informado,
      sentido,
      datetime_partida,
      datetime_chegada
    FROM
      rj-smtr-dev.SMTR202302003175_reprocessamento.viagem_completa
    WHERE 
      data IN ('2022-12-31','2023-01-01')
      AND SUBSTRING(id_veiculo, 2) IN ('41380','41125','41054','41464','41310','41011','41215','41265','41198','41021','41462','41278','41172','41244','41014','41010','41257','41313','41246','41026','41409','41377','41356','41396','41428','41111','41344','41104','41448','41372','41421','41092','41465','41247','41186','41435','41004','41030','41282','41391','41296','41209','41238','41202','41147','41442','41325','41322')
    


Downloading: 100%|██████████| 334/334 [00:00<00:00, 847.72rows/s]

Execução de query de viagens completas: sucesso.


In [13]:
viagem_completa_reprocessada = treat_trips(viagem_completa_reprocessada)
viagem_completa_amostra_reprocessada = check_trips(amostra, viagem_completa_reprocessada, "Viagem deferida")

print(viagem_completa_amostra_reprocessada.shape[0])
viagem_completa_amostra_reprocessada.to_excel('./../data/treated/viagem_completa_reprocessada.xlsx')

Não existem casos duplicados no cruzamento de dados.
181


In [14]:
viagem_conformidade_reprocessada = query_viagem_conformidade(data = datas, id_veiculo = id_veiculo, reprocessed=True,
                                                     reprocessed_table='rj-smtr-dev.SMTR202302003175_reprocessamento.viagem_conformidade')


  SELECT
    data,
    SUBSTRING(id_veiculo, 2) as id_veiculo,
    servico_informado,
    sentido,
    datetime_partida,
    datetime_chegada
  FROM
    rj-smtr-dev.SMTR202302003175_reprocessamento.viagem_conformidade
  WHERE 
    data IN ('2022-12-31','2023-01-01')
    AND SUBSTRING(id_veiculo, 2) IN ('41380','41125','41054','41464','41310','41011','41215','41265','41198','41021','41462','41278','41172','41244','41014','41010','41257','41313','41246','41026','41409','41377','41356','41396','41428','41111','41344','41104','41448','41372','41421','41092','41465','41247','41186','41435','41004','41030','41282','41391','41296','41209','41238','41202','41147','41442','41325','41322')
  


Downloading: 100%|██████████| 441/441 [00:00<00:00, 932.34rows/s]

Execução de query de viagens conformidade: sucesso.


In [15]:
viagem_conformidade_reprocessada = treat_trips(viagem_conformidade_reprocessada)

viagem_conformidade_amostra_reprocessada = check_trips(viagem_completa_amostra_reprocessada, viagem_conformidade_reprocessada, "Viagem indeferida por conformidade mesmo após o reprocessamento")
print(viagem_conformidade_amostra_reprocessada.shape[0])
viagem_conformidade_amostra_reprocessada.to_excel('./../data/treated/viagem_conformidade_reprocessada.xlsx')

Não existem casos duplicados no cruzamento de dados.
181


Checar viagens circulares

In [16]:
amostra_circular = circular_trips(viagem_conformidade_amostra_reprocessada, viagem_completa_reprocessada, viagem_conformidade_reprocessada)
amostra_circular.to_excel('./../data/treated/viagem_circular.xlsx')

Verificando se existem linhas circulares.


Downloading: 100%|██████████| 5/5 [00:00<00:00, 12.85rows/s]

Execução de query de viagens planejada: sucesso.
Empty DataFrame
Columns: [data, servico_amostra, id_veiculo_amostra, sentido_amostra, hora_inicio, hora_fim, datetime_partida_amostra, datetime_chegada_amostra, status, data_apurado, id_veiculo_apurado, servico_apurado, sentido_apurado, datetime_partida_apurado, datetime_chegada_apurado, circular_dividida]
Index: []
Não existem viagens circulares divididas em shapes de ida e volta.


Checar GPS

In [17]:
viagens_gps_reprocessada = gps_data(amostra_circular)
viagens_gps_reprocessada.to_excel('./../data/treated/viagem_gps_reprocessada.xlsx')

Downloading: 100%|██████████| 1374/1374 [00:00<00:00, 3375.95rows/s]

Execução de query de posições de GPS: sucesso.
Acesso aos sinais de GPS concluído com sucesso.
Tratamento de dados de GPS concluído com sucesso.


Checar ponto inicial e final

In [18]:
viagens_gps_reprocessadas_start_end = check_start_end_gps(viagens_gps_reprocessada)
viagens_gps_reprocessadas_start_end.to_excel('./../data/treated/viagem_gps_reprocessada_startend.xlsx')

Downloading: 100%|██████████| 3/3 [00:00<00:00,  7.52rows/s]


Execução de query de viagens planejada: sucesso.


Downloading: 100%|██████████| 682/682 [00:00<00:00, 1623.81rows/s]


Execução de query de posições de GPS: sucesso.
181
           data servico_amostra id_veiculo_amostra sentido_amostra  \
0    2022-12-31             309              41344           VOLTA   
1    2022-12-31             309              41465           VOLTA   
2    2022-12-31             309              41344           VOLTA   
3    2022-12-31             309              41465           VOLTA   
4    2022-12-31             309              41265           VOLTA   
..          ...             ...                ...             ...   
176  2023-01-01             309              41322           VOLTA   
177  2022-12-31             309              41380           VOLTA   
178  2022-12-31             309              41125           VOLTA   
179  2022-12-31             309              41247           VOLTA   
180  2022-12-31             309              41186           VOLTA   

    hora_inicio   hora_fim datetime_partida_amostra datetime_chegada_amostra  \
0      15:25:00   16:41:00  

In [19]:
print(viagens_gps_reprocessadas_start_end['status'].value_counts())

Viagem deferida                                                                    172
Sinal de GPS encontrado para o veículo operando em serviço diferente da amostra      4
Sinal de GPS encontrado para o veículo operando no mesmo serviço da amostra          3
Sinal de GPS não encontrado para o veículo no horário da viagem                      1
O veículo não passou no raio de 500m do ponto de partida/final do trajeto            1
Name: status, dtype: int64


Verificar os três casos em que o GPS foi encontrado para o mesmo veículo da amostra

In [20]:
q = f"""
SELECT
 *
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-12-31" AND servico = '309'
""" 
shape_planejado = bd.read_sql(q, from_file=True)
shape_planejado = shape_planejado.iloc[[1, 2]]
shape_planejado

Downloading: 100%|██████████| 3/3 [00:00<00:00,  6.05rows/s]


,data,tipo_dia,servico,vista,consorcio,sentido,distancia_planejada,distancia_total_planejada,inicio_periodo,fim_periodo,trip_id_planejado,trip_id,shape_id,shape_id_planejado,data_shape,shape,sentido_shape,start_pt,end_pt
1,2022-12-31,Sabado,309,Alvorada ↔ Central,Transcarioca,I,35.868,7327.146,10:00:00,22:00:00,O0309AAA0AIDU01,O0309AAA0AIDU01,O0309AAA0AIDU01,O0309AAA0AIDU01,2022-12-31,LINESTRING(-43.3659682760486 -23.0015126993213...,I,POINT(-43.3659682760486 -23.0015126993213),POINT(-43.1919224253361 -22.9044931043239)
2,2022-12-31,Sabado,309,Alvorada ↔ Central,Transcarioca,V,36.055,7327.146,03:20:00,22:45:00,O0309AAA0AVDU01,O0309AAA0AVDU01,O0309AAA0AVDU01,O0309AAA0AVDU01,2022-12-31,LINESTRING(-43.1919129747216 -22.9044908950373...,V,POINT(-43.1919129747216 -22.9044908950373),POINT(-43.3659585238186 -23.0015125015223)


In [21]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2022-12-31'
  AND SUBSTRING(id_veiculo, 2) = "41186"
  AND timestamp_gps BETWEEN "2022-12-31T19:29:00"
  AND "2022-12-31T20:38:00"
"""     
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()


Downloading: 100%|██████████| 116/116 [00:00<00:00, 328.61rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           116 non-null    object        
 1   servico              116 non-null    object        
 2   timestamp_gps        116 non-null    datetime64[ns]
 3   posicao_veiculo_geo  116 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.8+ KB


In [22]:
map = create_trip_map(dados_gps, shape_planejado)
map.save('./../data/figures/C41380_11h23m_31_12_2022.html')
map

Fazer correções nas três viagens do dia 31/12/2022:
- 11:23 - 12:41 41380 - foi paga após o reprocessamento cruzamento de 10 min não identificou (foi em 11 min).
- 11:36 - 12:52 41125 - foi paga após o reprocessamento cruzamento de 10 min não identificou (foi em 11 min).
- 19:29 - 20:38 41186 - desviou do trajeto (vide mapa acima).

In [30]:
viagens_gps_reprocessadas_start_end

,data,servico_amostra,id_veiculo_amostra,sentido_amostra,hora_inicio,hora_fim,datetime_partida_amostra,datetime_chegada_amostra,status,data_apurado,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2022-12-31,309,41344,VOLTA,15:25:00,16:41:00,2022-12-31 15:25:00,2022-12-31 16:41:00,Sinal de GPS encontrado para o veículo operand...,NaN,NaN,LECD37,NaN,NaT,NaT
1,2022-12-31,309,41465,VOLTA,16:15:00,17:34:00,2022-12-31 16:15:00,2022-12-31 17:34:00,Sinal de GPS encontrado para o veículo operand...,NaN,NaN,SP315,NaN,NaT,NaT
2,2022-12-31,309,41344,VOLTA,18:39:00,19:59:00,2022-12-31 18:39:00,2022-12-31 19:59:00,Sinal de GPS encontrado para o veículo operand...,NaN,NaN,LECD37,NaN,NaT,NaT
3,2022-12-31,309,41465,VOLTA,19:46:00,21:37:00,2022-12-31 19:46:00,2022-12-31 21:37:00,Sinal de GPS encontrado para o veículo operand...,NaN,NaN,SP315,NaN,NaT,NaT
4,2022-12-31,309,41265,VOLTA,23:45:00,00:47:00,2022-12-31 23:45:00,2022-12-31 00:47:00,Sinal de GPS não encontrado para o veículo no ...,NaN,NaN,,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2023-01-01,309,41322,VOLTA,20:00:00,21:13:00,2023-01-01 20:00:00,2023-01-01 21:13:00,Viagem deferida,2023-01-01,41322,309,V,2023-01-01 20:02:40,2023-01-01 21:11:10
177,2022-12-31,309,41380,VOLTA,11:23:00,12:41:00,2022-12-31 11:23:00,2022-12-31 12:41:00,Sinal de GPS encontrado para o veículo operand...,NaN,NaN,309,NaN,NaT,NaT
178,2022-12-31,309,41125,VOLTA,11:36:00,12:52:00,2022-12-31 11:36:00,2022-12-31 12:52:00,Sinal de GPS encontrado para o veículo operand...,NaN,NaN,309,NaN,NaT,NaT
179,2022-12-31,309,41247,VOLTA,16:48:00,18:56:00,2022-12-31 16:48:00,2022-12-31 18:56:00,O veículo não passou no raio de 500m do ponto ...,NaN,NaN,309,NaN,NaT,NaT


In [33]:
# Atualizando o status para 'Viagem deferida'
viagens_gps_reprocessadas_start_end.loc[(viagens_gps_reprocessadas_start_end['datetime_partida_amostra'] == '2022-12-31 11:23:00') & (viagens_gps_reprocessadas_start_end['id_veiculo_amostra'] == '41380'), 'status'] = 'Viagem deferida'
viagens_gps_reprocessadas_start_end.loc[(viagens_gps_reprocessadas_start_end['datetime_partida_amostra'] == '2022-12-31 11:36:00') & (viagens_gps_reprocessadas_start_end['id_veiculo_amostra'] == '41125'), 'status'] = 'Viagem deferida'

# Atualizando o status para 'Veículo desviou do trajeto previsto'
viagens_gps_reprocessadas_start_end.loc[(viagens_gps_reprocessadas_start_end['datetime_partida_amostra'] == '2022-12-31 19:29:00') & (viagens_gps_reprocessadas_start_end['id_veiculo_amostra'] == '41186'), 'status'] = 'Veículo desviou do trajeto previsto'

print(viagens_gps_reprocessadas_start_end['status'].value_counts())

Viagem deferida                                                                    174
Sinal de GPS encontrado para o veículo operando em serviço diferente da amostra      4
Sinal de GPS não encontrado para o veículo no horário da viagem                      1
O veículo não passou no raio de 500m do ponto de partida/final do trajeto            1
Veículo desviou do trajeto previsto                                                  1
Name: status, dtype: int64


In [34]:
viagens_gps_reprocessadas_start_end.to_excel('./../data/output/analise_amostra_pos_solucao.xlsx')